**Abstract.**
I reached the point in my machine learning journey where I understand the basic concepts
(backpropagation, regularization, CNNs, RNNs, and transformers), but wanted to dig deeper.
The machine learning engineers that I knew suggested that I train a model for a problem I
found interesting. I selected learning regular expressions from examples as a toy problem.
TODO

# Generating training data

My plan was:

1. Generate random regular expressions (regexes).
2. For each regex, generate some number of strings matching the regex.
3. For training data, let the $(x, y)$ input-output pairs be defined by:
   * $y$, a regex
   * $x$, a list of strings matching $y$, one per line
4. During training, reward if a forward pass through the network, starting with input $x$,
   produces an output $\hat{y}$ such that $y$ and $\hat{y}$ recognize the same strings.

## Generating random regular expressions

TODO

## Generating strings matching a regex

To generate strings matching a regex, I first converted the regex into a non-deterministic
finite automaton (NFA), then followed random paths through the NFA from an initial node to
a terminal node.

To work with regular expressions and finite automata, I started with
[Megha Bose's](https://github.com/Megha-Bose)
[Automata Theory Conversions](https://github.com/Megha-Bose/Automata-Theory-Conversions)
code. It wasn't licensed, so I asked her permission, and she said to go ahead.

### Walkthrough of Automata-Theory-Conversions

#### Step 1: rewrite regex (add concatenation, convert to postfix)

The first step in computing a regex to an NFA is to parse it. For this, it's useful to
rewrite the regex in a couple of ways:

1. Add concatenation symbols for convenience.
2. Convert the regex to postfix order.

In [1]:
# Union, Kleene star, concatenation, and grouping operations.
# All other characters are considered symbols
operators = ['+', '*', '.', '(', ')']


def add_concat(regex):
    """
    Adds concatenation operators ('.') to the input regex for convenience.
    
    Args:
      regex: a string representing the regular expression
      
    Returns:
      a list representing the regular expression with added concatenation operators
    """
    
    global non_symbols

    result = []
    for i in range(len(regex) - 1):
        result.append(regex[i])
        if regex[i] not in operators:
            if regex[i + 1] not in operators or regex[i + 1] == '(':
                result += '.'
        elif regex[i] == ')' and regex[i + 1] == '(':
            result += '.'
        elif regex[i] == '*' and regex[i + 1] == '(':
            result += '.'
        elif regex[i] == '*' and regex[i + 1] not in operators:
            result += '.'
        elif regex[i] == ')' and regex[i + 1] not in operators:
            result += '.'

    result += regex[-1]
    return result

In [2]:
print(add_concat("ab*"))
print(add_concat("(ab)*"))

['a', '.', 'b', '*']
['(', 'a', '.', 'b', ')', '*']


In [3]:
def compare_precedence(a, b):
    """
    Compares the precendence of the +, ., and * operators.
    
    Args:
      a: the first operator
      b: the second operator
      
    Returns:
      True iff a has precedence over b
    """
    p = ["+", ".", "*"]
    return p.index(a) > p.index(b)


def compute_postfix(regexp):
    """
    Converts an input regex to postfix order.
    
    Args:
      regex: list of regex operators and symbols
      
    Returns:
      list representing the regular expression in postfix order
    """
    
    stack = []
    result = []

    for c in regexp:
        if c not in operators or c == "*":
            result += c
        elif c == ")":
            while len(stack) > 0 and stack[-1] != "(":
                result += stack.pop()
            stack.pop()
        elif c == "(":
            stack.append(c)
        elif len(stack) == 0 or stack[-1] == "(" or compare_precedence(c, stack[-1]):
            stack.append(c)
        else:
            while len(stack) > 0 and stack[-1] != "(" and not compare_precedence(c, stack[-1]):
                result += stack.pop()
            stack.append(c)

    while len(stack) > 0:
        result += stack.pop()

    return result

In [4]:
print(compute_postfix(['a', '.', 'b']))
print(compute_postfix(['(', 'a', '.', 'b', ')']))
print(compute_postfix(['a', '.', 'b', '*']))
print(compute_postfix(['(', 'a', '.', 'b', ')', '*']))

['a', 'b', '.']
['a', 'b', '.']
['a', 'b', '*', '.']
['a', 'b', '.', '*']


#### Step 2: create expression tree from regex

Having converted the regex to postfix order, we create an expression tree from it. 

In [5]:
class OpType:
    """
    Constants for operator types.
    """
    SYMBOL = 1
    CONCAT = 2
    UNION  = 3
    KLEENE = 4


class ExpressionNode:
    """
    Node in an expression tree.
    """
    
    def __init__(self, op_type, value=None):
        self.op_type = op_type
        self.value = value
        self.left = None
        self.right = None

    def __str__(self):
        """
        Converts the node to a string in infix order, adding a lot of parens.
        """

        if self.op_type == OpType.SYMBOL:
            return self.value
        elif self.op_type == OpType.CONCAT:
            return '(' + self.left.__str__() + ').(' + self.right.__str__() + ')'
        elif self.op_type == OpType.UNION:
            return '((' + self.left.__str__() + ')+(' + self.right.__str__() + '))'
        elif self.op_type == OpType.KLEENE:
            return '(' + self.left.__str__() + ')*'
        else:
            raise Exception(f'unknown op type {self.op_type}')
        

def make_expression_tree(regexp):
    """
    Makes an expression tree from a regex.
    
    Args:
      regex: a regular expression in postfix form
      
    Returns:
      an expression tree
    """
    
    stack = []

    for c in regexp:
        if c == "+":
            z = ExpressionNode(OpType.UNION)
            z.right = stack.pop()
            z.left = stack.pop()
            stack.append(z)
        elif c == ".":
            z = ExpressionNode(OpType.CONCAT)
            z.right = stack.pop()
            z.left = stack.pop()
            stack.append(z)
        elif c == "*":
            z = ExpressionNode(OpType.KLEENE)
            z.left = stack.pop() 
            stack.append(z)
        elif c == "(" or c == ")":
            continue  # just for safety; our input does not contain parens
        else:
            stack.append(ExpressionNode(OpType.SYMBOL, c))
    return stack[0]

In [6]:
print(make_expression_tree(['a', 'b', '.']))
print(make_expression_tree(['a', 'b', '*', '.']))
print(make_expression_tree(['a', 'b', '.', '*']))

(a).(b)
(a).((b)*)
((a).(b))*


#### Step 3: convert expression tree into a NFA

Having parsed the regex into a tree, we now convert the tree into a nondeterministic finite automaton (NFA).

We use `$` to denote the empty string; in textbooks $\epsilon$ normally denotes the empty string.

In [17]:
class NFAState:
    def __init__(self):
        self.next_state = {}

def compute_nfa(node):
    """
    Compute the NFA for an ExpressionNode.
    
    Args:
      node: an ExpressionNode
      
    Returns:
      the start and end NFAState for node
    """
    
    if node.op_type == OpType.CONCAT:
        left_nfa  = compute_nfa(node.left)
        right_nfa = compute_nfa(node.right)

        # '$' represents the empty string (usually denoted epsilon)
        left_nfa[1].next_state['$'] = [right_nfa[0]]
        return left_nfa[0], right_nfa[1]

    elif node.op_type == OpType.UNION:
        start = NFAState()
        end = NFAState()

        first_nfa = compute_nfa(node.left)
        second_nfa = compute_nfa(node.right)

        start.next_state['$'] = [first_nfa[0], second_nfa[0]]
        first_nfa[1].next_state['$'] = [end]
        second_nfa[1].next_state['$'] = [end]

        return start, end

    elif node.op_type == OpType.KLEENE:
        start = NFAState()
        end = NFAState()

        starred_nfa = compute_nfa(node.left)

        start.next_state['$'] = [starred_nfa[0], end]
        starred_nfa[1].next_state['$'] = [starred_nfa[0], end]
        
        return start, end

    elif node.op_type == OpType.SYMBOL:
        start = NFAState()
        end = NFAState()
        start.next_state[node.value] = end
        return start, end
    
    else:
        raise Exception(f'unknown op type {node.op_type}')

#### Putting steps 1-3 together

We now wrap up steps 1-3 in a function that takes a regex as argument and returns an NFA.

In [18]:
def regex_to_nfa(regex):
    """
    Compute the NFA for a regular expression.
    
    Args:
      regex: a regular expression
    
    Returns:
      the start and end NFAState
    """
    
    concat = add_concat(regex)
    postfix = compute_postfix(concat)
    tree = make_expression_tree(postfix)
    start, end = compute_nfa(tree)
    return start, end

In [19]:
print(regex_to_nfa("ab*"))
print(regex_to_nfa("(ab)*"))

(<__main__.NFAState object at 0x7f357415c460>, <__main__.NFAState object at 0x7f357415cc40>)
(<__main__.NFAState object at 0x7f357415cdf0>, <__main__.NFAState object at 0x7f357415cf70>)


### Random paths through an NFA

Starting with an NFA, we've generated a regex. The next step is to generate strings matching
the regex from the NFA. To do that, we follow a random path through the NFA from the initial
node. We build the sequence of edge labels we see along the path traveled so far. When we reach
the terminal node, we output the edge labels we saw.

In [9]:
# TODO

TODO detail on equivalence of regular expressions